## 4. Create and Upload Model Archive

In [ ]:
import tarfile
import os

def compress(tar_dir=None, output_file="model.tar.gz"):
    parent_dir = os.getcwd()
    os.chdir(tar_dir)
    with tarfile.open(os.path.join(parent_dir, output_file), "w:gz") as tar:
        for item in os.listdir('.'):
            print(item)
            tar.add(item, arcname=item)
    os.chdir(parent_dir)

compress(str(model_tar))

from sagemaker.s3 import S3Uploader

s3_model_uri = S3Uploader.upload(local_path="model.tar.gz", desired_s3_uri=f"s3://{sess.default_bucket()}/sdxl-model")

print(f"Model uploaded to: {s3_model_uri}")

## 5. Deploy the Model

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

role = "arn:aws:iam::YOUR_ACCOUNT_ID:role/YOUR_SAGEMAKER_EXECUTION_ROLE"

huggingface_model = HuggingFaceModel(
    model_data=s3_model_uri,
    role=role,
    transformers_version="4.28.1",
    pytorch_version="2.0.0",
    py_version='py310',
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge"  # or an appropriate GPU instance
)

print(f"Endpoint name: {predictor.endpoint_name}")